## Использование баз данных в Питоне

Если вы хотите повторить код, который написан в этой тетрадке, то предполагается, что у вас на локальной машине стоит MySQL-сервер (я буду использовать Docker) и MySQL-клиент для Питона.

Если вы предпочитаете работать через Docker, то можно использовать следующую команду.

`docker run --rm -p 33061:3306 -e MYSQL_ROOT_PASSWORD=qwertbvcxz mysql`
- run - запустить;
- --rm - удалить контейнер после завершения;
- -p 33061:3306 - всю информацию с порта 3306 контейнера передавать на порт 33061 компьютера и наоборот.
- -e MYSQL_ROOT_PASSWORD=qwertbvcxz - передать параметр `MYSQL_ROOT_PASSWORD` со значением `qwertbvcxz` в контейнер (установи мне пароль от рута в значение qwertbvcxz);
- mysql - какой контейнер запускать.

Если у вас уже установлен MySQL сервер, то можно работать с ним через порт 3306 (по умолчанию). Так как я планирую работать через Docker, то и порт у меня будет установлен 33061 (указанный в командной строке).

Создание базы я буду делать через MySQL Workbench (скачивается [отсюда](https://dev.mysql.com/downloads/workbench/)).

Чтобы поставить себе библиотеки для Питона, работающие с MySQL, надо выполнить команду
`pip3 install mysql-connector-python`

In [1]:
import re

In [2]:
# Если не хочется долгих экспериментов, в папке лежит сокращенная версия файла с новостями:
# summer22.txt
with open("data/lenta2018_summer22.txt", encoding="utf-8") as newsfile: # Файл с новостями.
    text_news = [(n.split("-----\n")[0].split('\n')[0], 
                  n.split("-----\n")[0].split('\n')[1], 
                  n.split("-----\n")[1]) for n in newsfile.read().split("=====\n")[1:]]
    

In [3]:
text_news[:5]

[('«Королева дерьмовых роботов» перенесла операцию на мозге',
  '2018/06/01',
  'Шведская YouTube-знаменитость Симона Герц (Simone Giertz), прославившаяся необычными изобретениями, перенесла операцию. Об этом рассказали ее представители в Twitter. Еще в конце апреля девушка сообщила подписчикам о недуге. Оказалось, что у нее в мозге обнаружили доброкачественную опухоль размером с мяч для гольфа. Блогер даже дала ей имя, назвав новообразование Брайаном.  Герц сильно переживала перед хирургическим вмешательством. В одном из предоперационных видео она говорила, что очень напугана. Однако врачи быстро удалили опухоль и спасли девушке жизнь. «Симона вышла из хирургии, и ее врачи очень довольны тем, как все прошло. Она проспала достаточно долго, чтобы сделать неуместную шутку, так что все хорошо», — говорится на ее странице в Twitter.  27-летняя изобретательница из Стокгольма прославилась несколько лет назад, показывая подписчикам свои бесполезные изобретения. Например, Герц демонстрировала 

Теперь подключим библиотеку для работы с СУБД.

In [4]:
import mysql.connector

Для того, чтобы подключиться к базе данных создаем соедиение при помощи метода `connect`. В него необходимо передать следующие параметры: адрес (или название) сервера, на котором стоит СУБД; базу данных (схему), с которой мы собираемся работать; логин пользователя, имеющего право работать с этой базой; его пароль. 

Здесь я не указываю к какой базе мы подключаемся, так как считаю, что этой базы еще нет.

In [5]:
con = mysql.connector.connect(host='127.0.0.1', port=33061, user='root', password='qwertbvcxz')

Объект con теперь позволяет работать с соединением. С его помощью создадим курсор, который будет отправлять SQL-запросы и получать их результаты. Параметр dictionary=True означает, что результаты будут возвращаться в виде словаря. По умолчанию его значение равно False, то есть результаты будут возвращаться в виде списка. 

В первом случае выборка из базы данных будет возвращаться в виде списока записей, каждая из которых представляется словарем. Ключами в этом словаре будут имена полей, к которым будут привязаны соответствующие значения. Во втором случае получится список списков.

In [6]:
cur=con.cursor(dictionary=True)

Создадим базу `texts4experiments`, в которую будем складывать информацию о текстах.

In [7]:
cur.execute("CREATE DATABASE texts4experiments /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci */ /*!80016 DEFAULT ENCRYPTION='N' */;")

Теперь подключился к базе еще раз, указав название базы данных, которую мы будем использовать по умолчанию.

In [8]:
con = mysql.connector.connect(host='127.0.0.1', port=33061, database='texts4experiments', user='root', password='qwertbvcxz')

In [23]:
cur=con.cursor(dictionary=True)

In [10]:
cur.execute("""CREATE TABLE text_table (
  id_text INT NOT NULL AUTO_INCREMENT,
  header VARCHAR(256) NULL,
  article VARCHAR(9192) NULL,
  art_date VARCHAR(10) NULL,
  PRIMARY KEY (id_text),
  UNIQUE INDEX id_text_UNIQUE (id_text ASC) VISIBLE);
""")


Теперь нам надо загрузить новости в таблицу базы данных. Перебираем все заметки и вставвляем их в таблицу при помощи <a href="https://ru.wikipedia.org/wiki/Insert_(SQL)">INSERT</a>. После каждого оператора вызываем операцию подтверждения транзакции. Сама транзакция открывается автоматически при отправке SQL-запроса на сервер (если не была открыта ранее вручную при помощи метода <a href="https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-start-transaction.html">start_transaction()</a>). Все присылаемые данные СУБД не записывает в таблицу, а кеширует. Если в какой-то момент мы решим, что все действия с начала транзакции необходимо отменить, необхододимо вызвать метод rollback. Если все изменения должны быть применены удачно, требуется вызвать commit. Мы будем вызывать commit после каждого SQL-запроса, хотя сервер будет работать быстрее, если мы будем вызывать его, например, раз в десять запросов.<br>
В выводе можно увидеть запросы, которые не могут быть выполнены. Заметьте, что мы заменили все апострофы в тексте на \' чтобы они не считались СУБД как конец строки, добавляемой в базу. Однако текст содержит символы, которые СУБД воспринимает как нечитаемые. Чтобы побороть этот факт можно закодировать текст в кодировку <a href="https://docs.python.org/2/library/base64.html">Base64</a>, а при получении данных из СУБД раскодировать их обратно (или кодировать только проблемные строки, добавляя в начало определенный префикс).<br>
Но мы просто заменим символ апострофа ' на \\'.

SQL-запрос можно формировать динамически - это всего лишь строка.

In [11]:
%%time
for a in text_news[:1000]:
    
    head = a[0].replace("'", "\\'")
    art = a[2].replace("'", "\\'")
    head = a[0].replace('"', '\\"')
    art = a[2].replace('"', '\\"')
    
    try:
        #sss = "INSERT INTO text_table (header, article, art_date) VALUES ('"+head+"','"+art+"', '"+a[1]+"')"
        sss = f"INSERT INTO text_table (header, article, art_date) VALUES ('{head}', '{art}', '{a[1]}')"
        cur.execute(sss)
        #print(sss)
    except:
        print("Error - INSERT INTO text_table (header, article, art_date) VALUES ('"+head+"','"+art+"', '"+a[1]+"')")
    
    con.commit()

Error - INSERT INTO text_table (header, article, art_date) VALUES ('Ubisoft анонсировала новый Assassin’s Creed','Компания Ubisoft показала тизер новой игры серии Assassin's Creed. Ролик опубликован на официальной странице игры в Twitter. Незадолго до анонса в сети появилось изображение брелока со шлемом воина и подписью Assassin's Creed Odyssey. По информации Kotaku, действие игры будет разворачиваться в Древней Греции, кроме того, будет возможность выбрать между двумя игровыми персонажами. Ожидается, что игра выйдет до 31 марта 2019 года, релиз может состояться осенью 2018-го. Ubisoft подробно расскажет об игре на собственной пресс-конференции на E3. Она пройдет 11 июня, начало в 23:00 по московскому времени.
', '2018/06/01')
Error - INSERT INTO text_table (header, article, art_date) VALUES ('На ВДНХ показали модные платья','В Москве в павильоне выставочного комплекса ВДНХ прошло дефиле коллекции модного дома Chanel «Париж — Гамбург» Métiers d’art 2017/18, сообщает корреспондент «Лен

In [12]:
len(text_news)

224

Теперь попробуем прочитать записанные данные. С оператором <a href="https://ru.wikipedia.org/wiki/Select_(SQL)">SELECT</a> мы уже знакомы. <i>LIMIT 0, 10</i> говорит СУБД, что вернуть надо 10 записей начиная с 0-й. При следующем запросе можно попросить вернуть следующую порцию записей (например, <i>LIMIT 0, 10</i> вернет 10 записей начиная с 10-й).

In [24]:
cur.execute("SELECT * FROM text_table LIMIT 0, 20")

Теперь необходимо получить данные с сервера. <a href="https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-fetchall.html">fetchall</a> получает все результаты запроса. В качестве альтернативы можно использовать получение записей по одной - <a href="https://dev.mysql.com/doc/connector-python/en/connector-python-api-cext-fetch-row.html">fetch_row</a>.

In [25]:
data=cur.fetchall()

Посмотрим что у нас получилось в результате запроса. Да, в самом деле первые 10 добавленных статей.<br>
Ура! Мы умеем сохранять данные в СУБД и получать их обратно.

In [26]:
data

[{'id_text': 1,
  'header': '«Королева дерьмовых роботов» перенесла операцию на мозге',
  'article': 'Шведская YouTube-знаменитость Симона Герц (Simone Giertz), прославившаяся необычными изобретениями, перенесла операцию. Об этом рассказали ее представители в Twitter. Еще в конце апреля девушка сообщила подписчикам о недуге. Оказалось, что у нее в мозге обнаружили доброкачественную опухоль размером с мяч для гольфа. Блогер даже дала ей имя, назвав новообразование Брайаном.  Герц сильно переживала перед хирургическим вмешательством. В одном из предоперационных видео она говорила, что очень напугана. Однако врачи быстро удалили опухоль и спасли девушке жизнь. «Симона вышла из хирургии, и ее врачи очень довольны тем, как все прошло. Она проспала достаточно долго, чтобы сделать неуместную шутку, так что все хорошо», — говорится на ее странице в Twitter.  27-летняя изобретательница из Стокгольма прославилась несколько лет назад, показывая подписчикам свои бесполезные изобретения. Например, 

Добавим новости в словарь пакетами.

In [27]:
%%time
data=[]
for a in text_news[:1000]:
    head = a[0].replace("'", "\\'")
    art = a[2].replace("'", "\\'")
    data.append("('{}','{}','{}')".format(a[0].replace("'", "\\'"), a[2].replace("'", "\\'"), a[1]))

for i in range(3):
    req="INSERT INTO text_table (header, article, art_date) VALUES " + ",".join(data[i*100:(i+1)*100])
    cur.execute(req)
con.commit()

CPU times: user 8.7 ms, sys: 14 µs, total: 8.71 ms
Wall time: 183 ms


Создадим словарь токенов и их частот.

In [37]:
news_dict = []
for art in text_news:
    words = re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", art[2])
    words = list([w[0].lower() for w in words])
    news_dict.extend(words)
    
news_dict = list(set(news_dict))

In [38]:
len(news_dict)

12233

In [29]:
# tqdm красиво умеет показывать прогресс вычислений.
from tqdm.auto import tqdm

In [31]:
cur.execute("""CREATE TABLE news_freq (
  id_token INT NOT NULL AUTO_INCREMENT,
  token VARCHAR(256) NULL,
  freq INT NULL,
  PRIMARY KEY (id_token),
  UNIQUE INDEX id_token_UNIQUE (id_token ASC) VISIBLE);
""")


In [39]:
data = []
cntr = 0
cntr2 = 0
for a in tqdm(news_dict):
    data.append(f"('{a}', 0)")
    cntr += 1
    if cntr == 100:
        req="INSERT INTO news_freq (token, freq) VALUES " + ",".join(data)
        cur.execute(req)
        cntr = 0
        data = []
        cntr2 += 1
        if cntr2 == 10:
            con.commit()
            cntr2 = 0
            
if len(data) != 0:
    req="INSERT INTO news_freq (token, freq) VALUES " + ",".join(data)
    cur.execute(req)
con.commit()
    

Посчитаем частоты слов в документах или всего в корпусе.

In [40]:
from collections import Counter

In [41]:
news_dict = []
for art in text_news:
    words = re.findall("[А-ЯЁа-яё]+", art[2])
    words = list([w.lower() for w in words])
    news_dict.extend(words)
    
#news_dict = list(set(news_dict))

freq_news = Counter(news_dict)

In [42]:
freq_news

Counter({'шведская': 1,
         'знаменитость': 1,
         'симона': 2,
         'герц': 4,
         'прославившаяся': 1,
         'необычными': 1,
         'изобретениями': 1,
         'перенесла': 7,
         'операцию': 6,
         'об': 191,
         'этом': 187,
         'рассказали': 7,
         'ее': 64,
         'представители': 10,
         'в': 1770,
         'еще': 39,
         'конце': 3,
         'апреля': 13,
         'девушка': 6,
         'сообщила': 10,
         'подписчикам': 2,
         'о': 164,
         'недуге': 1,
         'оказалось': 3,
         'что': 500,
         'у': 72,
         'нее': 5,
         'мозге': 1,
         'обнаружили': 9,
         'доброкачественную': 1,
         'опухоль': 3,
         'размером': 1,
         'с': 448,
         'мяч': 2,
         'для': 108,
         'гольфа': 1,
         'блогер': 4,
         'даже': 14,
         'дала': 1,
         'ей': 13,
         'имя': 10,
         'назвав': 1,
         'новообразование': 1,
         

In [43]:
data = []
cntr = 0
cntr2 = 0
for t, f in tqdm(freq_news.items()):
    data.append(f"('{t}', {f})")
    cntr += 1
    if cntr == 100:
        req="INSERT INTO news_freq (token, freq) VALUES " + ",".join(data)
        cur.execute(req)
        cntr = 0
        data = []
        cntr2 += 1
        if cntr2 == 10:
            con.commit()
            cntr2 = 0
            
if len(data) != 0:
    req="INSERT INTO news_freq (token, freq) VALUES " + ",".join(data)
    cur.execute(req)
con.commit()
    

In [ ]:
# Удалим слова, которые обавили вначале с нулевой частотой.
cur.execute("DELETE FROM news_freq WHERE freq=0")

In [49]:
cur.execute("SELECT count(*) as cnt, avg(freq) as avg_freq FROM texts4experiments.news_freq")
res = cur.fetchall()
print(f"Количество записей:{res[0]['cnt']}, средняя частота токена {res[0]['avg_freq']}")

Количество записей:12227, средняя частота токена 2.9851


Посмотрим как выбираются статьи из базы.

In [50]:
cur.execute("SELECT id_text, article FROM text_table")
news_from_db = cur.fetchall()

In [51]:
news_from_db[:3]

[{'id_text': 1,
  'article': 'Шведская YouTube-знаменитость Симона Герц (Simone Giertz), прославившаяся необычными изобретениями, перенесла операцию. Об этом рассказали ее представители в Twitter. Еще в конце апреля девушка сообщила подписчикам о недуге. Оказалось, что у нее в мозге обнаружили доброкачественную опухоль размером с мяч для гольфа. Блогер даже дала ей имя, назвав новообразование Брайаном.  Герц сильно переживала перед хирургическим вмешательством. В одном из предоперационных видео она говорила, что очень напугана. Однако врачи быстро удалили опухоль и спасли девушке жизнь. «Симона вышла из хирургии, и ее врачи очень довольны тем, как все прошло. Она проспала достаточно долго, чтобы сделать неуместную шутку, так что все хорошо», — говорится на ее странице в Twitter.  27-летняя изобретательница из Стокгольма прославилась несколько лет назад, показывая подписчикам свои бесполезные изобретения. Например, Герц демонстрировала самодельный будильник, к которому приделана резинов

Создадим таблицу, которая хранит в каком документе какое слово сколько раз встретилось.

In [60]:
cur.execute("DROP TABLE IF EXISTS text2token")

In [61]:
cur.execute("""CREATE TABLE text2token (
  id_tt INT NOT NULL AUTO_INCREMENT,
  token_id VARCHAR(256) NULL,
  text_id INT NULL,
  freq INT NULL,
  PRIMARY KEY (id_tt),
  UNIQUE INDEX id_tt_UNIQUE (id_tt ASC) VISIBLE);
""")


In [62]:
for news in tqdm(news_from_db):
    words = re.findall("[А-ЯЁа-яё]+", news['article'])
    words = list([w.lower() for w in words])
    wrd_freqs = Counter(words)
    text_id = news['id_text']
    
    for w, f in wrd_freqs.items():
        cur.execute(f"SELECT id_token FROM news_freq WHERE token='{w}'")
        wid = cur.fetchall()
        if len(wid) == 1:
            tid = wid[0]['id_token']
            cur.execute(f"INSERT INTO text2token (token_id,text_id,freq) VALUES ({tid},{text_id},{f})")
            
con.commit()
            
